##### Definicion de lenet5 by Dive into deep learning
https://d2l.ai/chapter_convolutional-neural-networks/lenet.html

### Recursos tenidos en cuenta para implementar el modelo
(contiene implementacion del modelo)

https://d2l.ai/chapter_linear-classification/classification.html

(contiene implementacion del modelo)

https://blog.paperspace.com/writing-lenet5-from-scratch-in-python/ 

##### Tutorial oficial pytorch
https://pytorch.org/tutorials/beginner/basics/intro.html 

Clase Sequential

https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html?highlight=sequential#torch.nn.Sequential

Capa Conv2d 

https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html




In [1]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [2]:
from numpy import pad
import torch
from torch import nn
class Lenet5(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        # se reciben imagenes 3x32x32
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, stride=1, padding=0), # quedan un volumen de 6x28x28 # (n - k + s + p)/s
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2) # queda un volumen de 6x14x14
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0), # queda un volumen de 16x10x10
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2) # queda un volumen de 16x5x5 = 400.
        )

        self.denseLayer = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=400, out_features=120),
            nn.ReLU(),
            nn.Linear(in_features=120, out_features=84),
            nn.ReLU(),
            nn.Linear(in_features=84, out_features=10)
        )
    
    def forward(self, x):
        logits = self.layer1(x)
        logits = self.layer2(logits)
        logits = self.denseLayer(logits)
        # no es necesario utilizar softmax, ya que el optimizador lo integra, pero es necesario agregarlo por afuera para cuando se utilice el modo de evaluacion.
        return logits


In [6]:
random = torch.rand(1,3,32,32)
model = Lenet5()
model.eval()
print(model(random))    

tensor([[-0.0596, -0.0419,  0.0022,  0.0457,  0.0425, -0.0692,  0.1201, -0.0016,
         -0.1000,  0.0421]], grad_fn=<AddmmBackward0>)
